# Predict low, high strategy  EDA

To install conda and binance packages to this notebook uncomment the code below

In [1]:
%%capture
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
#mode="dev"
symbol="BTCUSDT"
mode="yc"
exchange="huobi"
#strategy="SimpleKerasStrategy"
strategy="LSTMStrategy2"

In [3]:

import os

# Downloading latest pytrade2 data
if mode == "yc":
    os.system("cd ./../deploy/yandex_cloud; ./download_data.sh")
    data_dir=f"../data/yandex-cloud"
else:
    data_dir=f"../data/dev"
    
print(f"Download completed. Local data dir: {data_dir}")

    

receiving incremental file list
LSTMStrategy/LSTMStrategy.db
LSTMStrategy/Xy/2023-06-23_BTC-USDT_X.csv
LSTMStrategy/Xy/2023-06-23_BTC-USDT_data.csv
LSTMStrategy/Xy/2023-06-23_BTC-USDT_y.csv
LSTMStrategy/account/2023-06-23_balance.csv
LSTMStrategy/weights/2023-06-23T07:49:05.342896.data-00000-of-00001
LSTMStrategy/weights/2023-06-23T07:49:05.342896.index
LSTMStrategy/weights/checkpoint
LSTMStrategy2/LSTMStrategy2.db
LSTMStrategy2/Xy/2023-06-22_BTC-USDT_X.csv
LSTMStrategy2/Xy/2023-06-22_BTC-USDT_data.csv
LSTMStrategy2/Xy/2023-06-22_BTC-USDT_y.csv
LSTMStrategy2/account/2023-06-22_balance.csv
LSTMStrategy2/account/2023-06-23_balance.csv
LSTMStrategy2/weights/2023-06-22T18:18:53.972783.data-00000-of-00001
LSTMStrategy2/weights/2023-06-22T18:18:53.972783.index
LSTMStrategy2/weights/2023-06-22T19:34:51.221881.data-00000-of-00001
LSTMStrategy2/weights/2023-06-22T19:34:51.221881.index
LSTMStrategy2/weights/checkpoint
LSTMStrategy2/weights/2023-06-22T18:19:51.815805_temp/part-00000-of-00001.data

## Init Huobi or Binance client

In [5]:
import os
import sys
import yaml
from binance.spot import Spot as Client
from huobi.client.account import AccountClient
from huobi.client.algo import AlgoClient
from huobi.client.market import MarketClient
from huobi.client.trade import TradeClient


def load_config_files(cfg_dir:str, paths:{})->{}:
    config={}
    for path in paths:
        fullpath=f"{cfg_dir}/{path}"
        with open(fullpath) as file:
            print("Reading config from "+fullpath)
            config.update(yaml.safe_load(file))
    return config

def load_config_local(strategy:str):
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    print("Reading local config")
    # Defaults
    cfg_dir="./../pytrade2/cfg"
    paths=["app-defaults.yaml", "app.yaml"]
    return load_config_files(cfg_dir, paths)

def load_config_yc(strategy):
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    strategy=strategy.lower()
    print("Reading yc config")
    project_dir="./.."
    cfg_dir=f"{project_dir}/deploy/yandex_cloud/secret/"
    paths=["app.yaml", f"{strategy}.yaml"]

    # Load yc config
    config = load_config_local(strategy)
    config.update(load_config_files(cfg_dir, paths))
    return config

def load_config(strategy):
    """ Read yc or dev config"""
    strategy = strategy.lower()
    return load_config_yc(strategy) if mode == "yc" else load_config_local(strategy)


def create_binance_client(strategy: str, mode: str):
        """ Binance client creation """
        print(mode)
        # Read configs to get keys from there
        strategy=strategy.lower()
        config = load_config(strategy)
        key,secret=config[f"pytrade2.exchange.{exchange}.connector.key"],config[f"pytrade2.exchange.{exchange}.connector.secret"],config[f"pytrade2.exchange.{exchange}.spot.url"]
        return Client(key=key, secret=secret, base_url=url, timeout=10)
#         key,secret,url=config[f"pytrade2.exchange.{exchange}.connector.key"],config[f"pytrade2.exchange.{exchange}.connector.secret"],config[f"pytrade2.exchange.{exchange}.spot.url"]
#         return Client(key=key, secret=secret, base_url=url, timeout=10)

def create_huobi_clients(config, strategy: str, mode: str):
    """ Huobi client creation """
    strategy=strategy.lower()
    key,secret=config[f"pytrade2.exchange.huobi.connector.key"],config[f"pytrade2.exchange.huobi.connector.secret"]
    return  MarketClient(api_key=key, secret_key=secret), \
            AccountClient(api_key=key, secret_key=secret), \
            TradeClient(api_key=key, secret_key=secret)


if exchange == "binance":
    binance_client = create_binance_client(strategy, mode)
    print("Created Binance client")
elif exchange == "huobi":
    config=load_config(strategy.lower())
    huobi_market_client, huobi_account_client, huobi_trade_client = create_huobi_clients(config, strategy, mode)
    huobi_account_id = config.get("pytrade2.broker.huobi.account.id")
    symbol=symbol.lower()

Reading yc config
Reading local config
Reading config from ./../pytrade2/cfg/app-defaults.yaml
Reading config from ./../pytrade2/cfg/app.yaml
Reading config from ./../deploy/yandex_cloud/secret//app.yaml
Reading config from ./../deploy/yandex_cloud/secret//lstmstrategy2.yaml


## Account and orders

### Account info

In [6]:
def print_binance_account():
    account=client.account()
    usdt_free=[a["free"] for a in account["balances"] if a["asset"]=="USDT"]
    # update_time=account["update_time"]
    # print(f"account update time:{update_time}")
    print(f"My USDT amount({strategy}): {usdt_free}")

    print(account)
    print(str(pd.to_datetime(account["updateTime"], unit="ms")))
# btc 1.15798200
#usdt 5134.30859706 26.04.2023 14:26
# usdt amount 
# 14.05.2023 9602.27USDT
# 14.05.2023 9613.27USDT
# 15.05.2023 12:11 10851.4496USDT
# 15.05.2023 15:26 USDT 10824.09131356, BTC 0.97100000
# 16.05.2023 19:12 USDT 10824.13392356 BTC 0.97100000
# 16.05.2023 22:09 USDT 10824.13237356, BTC 0.971
# 17.05.2023 0:47  USDT 10824.08461448 BTC 0.971
# 17.05.2023 9:44 USDT 10824.05857448 BTC 0.971
# 17.05.2023 17:22 USDT 10824.00134797 BTC 0.971
# 18.05.2023 08:25:81 USDT 10823.33222738 BTC 0.971
# 18.05.2023 23:09 USDT 10821.81933906 BTC 0.971
# 18.05.2023 07:22 USDT 10821.70723260 BTC 0.971 
# 19.05.2023 15:20 USDT 10820.82563494 locked 27.12356000 BTC 9.970
# 19.95.2023 15:25 USDT 10821.09770494 BTC 0.971

# Set sharp ratio=4
# 19.05.2023 19:36 USDT 10794.27474497 BTC 0.971
# 20.05.2023 15:35 USDT 10821.28357494 BTC 0.971
def print_huobi_account():
    print("Balance:")
    balances = huobi_account_client.get_balance(account_id=huobi_account_id)
    for b in filter(lambda b: b.balance != "0", balances):
        print(vars(b))
    
if exchange == "binance":
    print_binance_account()
elif exchange == "huobi":
    print_huobi_account()


Balance:


HuobiApiException: ('InputError', '[Input] account-id should not be null')

### Orders from Exchange

In [ ]:
def huobi_order_vars(order):
    """ Convert huobi order to it's attributes """
    order.created_at = pd.to_datetime(order.created_at, unit='ms')
    if order.finished_at: order.finished_at = pd.to_datetime(order.finished_at, unit='ms')
    if order.canceled_at: order.canceled_at = pd.to_datetime(order.canceled_at, unit='ms')
    return vars(order)

In [ ]:
def get_binance_orders():
    orders = pd.DataFrame(client.get_orders(symbol))
    for timecol in ["time", "updateTime", "workingTime"]:
        orders[timecol] = pd.to_datetime(orders[timecol], unit='ms')
    return orders.set_index("time", drop=False)

def get_huobi_orders():
    orders = huobi_trade_client.get_orders(symbol=symbol, order_state="filled")
    orders = pd.DataFrame([huobi_order_vars(o) for o in orders]).set_index("finished_at", drop=False)
    return orders

if exchange == "binance":
    get_binance_orders().tail()
elif exchange == "huobi":
    orders = get_huobi_orders()

if exchange == "binance":
    orders = get_binance_orders()
elif exchange == "huobi"    :
    orders = get_huobi_orders()

orders.tail()


### Opened orders from Exchange

In [ ]:
def get_binance_opened_orders():
    orders=pd.DataFrame(client.get_open_orders(symbol))

    for timecol in ["time", "updateTime", "workingTime"]:
        if timecol in orders.columns:
            orders[timecol] = pd.to_datetime(orders[timecol], unit='ms')
    if "time" in orders.columns:
        orders.set_index("time", drop=False, inplace=True)
    return order


    
def get_huobi_opened_orders():
    opened_orders = huobi_trade_client.get_open_orders(symbol=symbol, account_id = huobi_account_id)
    opened_orders = pd.DataFrame([huobi_order_vars(o) for o in opened_orders])
    if not opened_orders.empty: opened_orders = opened.orders.set_index("created_at").round(2)
    return opened_orders



if exchange == "binance":
    opened_orders = get_binance_opened_orders()
elif exchange == "huobi"    :
    opened_orders = get_huobi_opened_orders()

print("Opened orders")    
opened_orders.tail()


### My trades from binance

In [ ]:
def print_binance_trades():
    mytrades=pd.DataFrame(client.my_trades(symbol))
    mytrades["time"] = pd.to_datetime(mytrades["time"], unit='ms')
    mytrades.tail()

if exchange == "binance":
    print_binance_trades()

### Trades from bot database

In [ ]:
import pandas as pd
import numpy as np

def get_trades(data_dir: str, strategy: str, mode: str):
    import sqlite3
    db_path=f"{data_dir}/{strategy}/{strategy}.db"
    fee=0.0012
    print(f"Get trades from {db_path}")

    with sqlite3.connect(db_path) as conn:
        trades=pd.read_sql_query("SELECT * FROM trade ORDER BY open_time",conn,
                                 parse_dates=["open_time", "close_time"])#.set_index("open_time", drop=False)
    trades["profit"] = np.where(trades["side"] == "BUY", 
                                (trades["close_price"]-trades["open_price"])*trades["quantity"],
                                (trades["open_price"]-trades["close_price"])*trades["quantity"],
                               )
    trades["profit"] = trades["profit"] - fee*(trades["open_price"] + trades["close_price"])*trades["quantity"]
    trades["cum_profit"] = trades["profit"].cumsum()
    
    return trades

trades=get_trades(data_dir, strategy, mode)
trades.tail()

In [ ]:
print(f"Total profit: {trades['profit'].sum()}")


### Opened trades from trade bot database

In [ ]:
trades[trades.close_time.isna()].tail()

## Profit

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
sns.set_theme()
#paper, notebook, talk, poster

import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "plotly_mimetype+notebook"
import plotly.graph_objects as go

    
plot_figsize=(10,5)


def plot_profits_plotly(trades):
    """ Plot profit charts using plotly"""
    px.line(trades, x="open_time", y="profit", title=f"{strategy} profit").update_traces(mode='lines+markers').show()
    px.line(trades, x="open_time", y="cum_profit", title=f"{strategy} cumulative profit").update_traces(mode='lines+markers').show()

def plot_profits_sns(trades):
    """ Plot profit charts using sns"""
    plt.figure(figsize=plot_figsize)
    sns.set_style("darkgrid")
    sns.lineplot(data=trades, x="open_time", y="profit").set_title(f"{strategy} profit")
    plt.show()

    #sns.set_palette("deep")
    sns.set_style("whitegrid")
    plt.figure(figsize=plot_figsize)
    sns.lineplot(data=trades, x="open_time", y="cum_profit").set_title( f"{strategy} cumulative profit")
    plt.show()
    
    # Cumulative profit w/o outliers
    profit=trades[["open_time","profit"]].set_index("open_time")
    q_low = profit["profit"].quantile(0.1)
    q_hi  = profit["profit"].quantile(0.9)
    profit_filtered = profit[(profit["profit"] < q_hi) & (profit["profit"] > q_low)]
    cumsum_filtered = profit_filtered.cumsum()
    plt.figure(figsize=plot_figsize)
    sns.set_style("darkgrid")
    sns.lineplot(data=cumsum_filtered).set_title(f"{strategy} profit w/o outliers")
    plt.show()

def plot_profits_plt(trades):
    """ Plot profits using matplotlib """
    plt.figure(figsize=(10,5))
    sns.set_style("whitegrid")
    plt.plot("open_time", "profit", data=trades)
    plt.title(f"{strategy} profit")
    plt.show()
    
    sns.set_style("darkgrid")
    plt.figure(figsize=(10,5))
    plt.plot("open_time", "cum_profit", data=trades)
    plt.title(f"{strategy} cumulative profit")
    plt.show()

plot_profits_plotly(trades)
#plot_profits_sns(trades[:-1:])
#plot_profits_plt(trades)


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns


def plot_trade_counts(trades):
    buy_sell_counts = trades["side"].value_counts()
    total=len(trades)
    colors = sns.color_palette('pastel')[0:5]
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_size_inches(plot_figsize)
    fig.suptitle('Profit/loss trade counts')
    # Buy/sell counts
    ax1.pie([len(trades[trades["side"]=="BUY"]),len(trades[trades["side"]=="SELL"])], 
            labels=["BUY","SELL"], \
            autopct=lambda p: '{:.0f}'.format(p * total / 100))
    # Buy profit/loss counts
    ax2.pie([len(trades[(trades["side"]=="BUY") &  (trades["profit"]>0)].index),\
            len(trades[(trades["side"]=="BUY") &  (trades["profit"]<0)].index) \
            ],
            labels=["buy profit", " buy loss"], autopct="%.0f%%", colors=colors)
    # Sell profit/loss counts
    ax3.pie([len(trades[(trades["side"]=="SELL") &  (trades["profit"]>0)].index),\
            len(trades[(trades["side"]=="SELL") &  (trades["profit"]<0)].index) \
            ],
            labels=["sell profit", " sell loss"], autopct="%.0f%%", colors=colors)

    
plot_trade_counts(trades)



In [ ]:
import plotly.express as px

def plot_profit_stat_plotly(trades):
    """ Profit statistics: summary, average """
    buy_profit = trades.loc[trades["side"]=="BUY","profit"]
    sell_profit = trades.loc[trades["side"]=="SELL","profit"]
    px.bar(x=["buy_profit_sum", "sell_profit_sum"], y=[buy_profit.sum(), sell_profit.sum()], title="Buy/sell profit sum").show()
    px.bar(x=["buy_profit_mean", "sell_profit_mean"], y=[buy_profit.mean(), sell_profit.mean()], title="Buy/sell mean profit").show()

def plot_profit_stat_sns(trades):
    """ Profit statistics: summary, average """
    buy_profit = trades.loc[trades["side"]=="BUY","profit"]
    sell_profit = trades.loc[trades["side"]=="SELL","profit"]
    buy_profit_mean, sell_profit_mean = buy_profit.mean(), sell_profit.mean()
    buy_profit_mdn, sell_profit_mdn = buy_profit.median(), sell_profit.median()
    
    sns.barplot(x=["buy profit sum", "sell profit sum"], y = [buy_profit.sum(), sell_profit.sum()])
    plt.title("Profit sum")
    plt.show()
    #sns.set_style("whitegrid")
    
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(plot_figsize)
#     fig.align_ylabels(ax2)    

    data={"val": [buy_profit.mean(), sell_profit.mean(), buy_profit.median(), sell_profit.median()],
          "type": ["buy", "sell", "buy", "sell"],
          "agg": ["mean", "mean", "median", "median"]
         }
    sns.barplot(data=data, x="type", y="val", hue="agg")
    #sns.barplot(x=["buy profit median", "sell profit median"], y = [buy_profit.median(), sell_profit.median()],ax=ax2)
    #fig.show()
    
#     px.bar(x=["buy_profit_sum", "sell_profit_sum"], y=[buy_profit.sum(), sell_profit.sum()], title="Buy/sell profit sum").show()
#     px.bar(x=["buy_profit_mean", "sell_profit_mean"], y=[buy_profit.mean(), sell_profit.mean()], title="Buy/sell mean profit").show()
    
plot_profit_stat_sns(trades)

## Trades on candlestick chart

In [ ]:
from datetime import datetime
from huobi.model.market.candlestick import Candlestick

def get_last_candles_binance():
    candle_columns = ["open_time", "open", "high", "low", "close", "vol", "close_time", "quote_asset_volume",
                      "number_of_trades", " taker_buy_base_asset_volume", "taker_buy_quote_asset_volume",
                      "ignore"]    
    end_date=trades.open_time.max() if not trades.empty else datetime.utcnow()
    
    #end_time_millis=end_date.value//1000000
    cnt=24*60 # Number of last candles to read
    raw_candles = client.klines(symbol=symbol,
                                    interval="1m",
                                    #limit=10,
                                    limit=cnt,
                                    #endTime=end_time_millis
                               )
    df = pd.DataFrame(raw_candles, columns=candle_columns)
    df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
    df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
    df.set_index("open_time", drop=False, inplace=True)
        # Convert strings to float prices
    df[["open", "high", "low", "close"]] = df[["open", "high", "low", "close"]].astype(float)
    return df

def clean_candles_binance(df):
    """ Remove outliers """
    q_low = df["low"].diff().quantile(0.1)
    q_high  = df["high"].diff().quantile(0.9)
    df=df[df["high"].diff()< q_high]
    df=df[df["low"].diff() > q_low ]
    return df

def get_last_candles_huobi():
        """ Read candles from Huobi """
        # Get candles
        interval="1m"
        size=2000
        candles = huobi_market_client.get_candlestick(symbol=symbol, period="1min", size=size)
        candles = [vars(c) for c in candles]

        # Huobi candles don't have time, so generate times for last candles
        deltatime = pd.Timedelta(interval+"in")
        times = [datetime.utcnow() - deltatime * i for i in range(len(candles))]
        candles = pd.DataFrame(candles, index=times)
        candles["close_time"] = candles.index
        candles["open_time"] = candles["close_time"] - deltatime

        return candles.sort_index()

if exchange == "binance":
    candles = get_last_candles_binance()
    candles = clean_candles_binance(candles)
elif exchange == "huobi":
    candles = get_last_candles_huobi()

candles.tail()

In [ ]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd
import numpy as np
import datetime

def get_chart_figure(candles: pd.DataFrame, trades: pd.DataFrame)->pd.DataFrame:
    """
    Form plotly candlestick chart with buy/sell marks on it
    """
    
    df=candles.copy()
    #df=clean_candles(df)
    fig = go.Figure(data=[ \
                        go.Candlestick(\
                            x=df.index,\
                            open=df['open'],\
                            high=df['high'],\
                            low=df['low'],\
                            close=df['close'])
                         ])

    # Customize ay to distribute captions by y so they do not overlap by x
    ay,aymax,aystep=0,50,10
    #start_time=df.open_time.min()
    # Annotate buy, sell on the chart
    for i,trade in trades[(~trades["open_time"].isna()) & (trades["open_time"] > df.open_time.min())].iterrows():
        ay=(ay+aystep)%aymax
        color='green' if trade['side']=="BUY" else 'red'

        # Draw open order
        open_text='buy' if trade['side']=="BUY"  else 'sell'
        open_hover=f"{trade['open_time']} {open_text} at {str(trade['open_price'])}, sl={trade['stop_loss_price']}, tp={trade['take_profit_price']}"
        fig.add_annotation(x=trade['open_time'], \
                           y=trade['open_price'],\
                           text=open_text,\
                           hovertext=open_hover,\
                           showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                           font={'color':color}
                          )
    for i,trade in trades[(~trades.close_time.isna()) & (trades.close_time > df.close_time.min())].iterrows():
        color='green' if trade['side']=="BUY" else 'red'

        # Draw close order
        #profit = trade["close_price"]-trade["open_price"] if trade["close_price"] else None
        status= f", profit: {round(trade.profit, 10)}" if trade.profit else ""
        close_text=f"close {trade['side'].lower()}"
        close_hover=f"{trade['close_time']} {close_text} at {str(trade['close_price'])}{status}"
        fig.add_annotation(x=trade['close_time'], \
                           y=trade['close_price'],\
                           text=close_text,\
                           hovertext=close_hover,\
                           showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,\
                           font={'color':color}
                          )
    return fig

get_chart_figure(candles, trades).show()



## Predicted and actual low/high

In [ ]:
import os
import glob
import pandas as pd
from datetime import timedelta,date
import matplotlib.pyplot as plt
import plotly.express as px

def read_last_data(data_dir, strategy):
    """ Read days from start to end from local predictions history"""
    
    def read_last(symbol: str, name: str, n: int):
        xy_dir=f"{data_dir}/{strategy}/Xy"
        file_path = sorted([f for f in os.listdir(xy_dir) if f.endswith(f"{symbol}_{name}.csv")])[-1]
        file_path = f"{xy_dir}/{file_path}"
        print(f"Read from {file_path}")
        return pd.read_csv(file_path, parse_dates=True, index_col="datetime").tail(n)
    n=1000
    data=read_last("BTCUSDT", "data", n)
    X=read_last("BTCUSDT", "X", n)
    y=read_last("BTCUSDT", "y", n)
    data=pd.merge(data[["symbol"]],y, left_index=True, right_index=True) 
    #y=pd.concat([data,y])
    return data,X,y


# Read candles for last 3 days
data,x,y = read_last_data(data_dir, strategy)
data.tail(100)

In [ ]:

def get_predictions(data):
    """ 
    Merge predicted and real prices for analysis 
    """
    
    # Prediction window is constant, get it from last row
    predicted=data[["bid","ask","bid_min_fut", "bid_max_fut", "ask_min_fut", "ask_max_fut"]]
    predicted.columns=[c.replace("_fut", "_predicted") for c in predicted.columns]
    predict_window=data["predict_window"].tail(1).values[0]
    real_min=data[["bid", "ask"]].rolling(predict_window).min().rename(columns={"bid":"bid_min_real", "ask":"ask_min_real"})
    real_max=data[["bid", "ask"]].rolling(predict_window).max().rename(columns={"bid":"bid_max_real", "ask":"ask_max_real"})
    real=pd.concat([real_min, real_max], axis=1)
    real["prediction_datetime"]=real.index.to_series()-pd.to_timedelta(predict_window)
    merged=pd.merge_asof(real, predicted, left_on="prediction_datetime", right_index=True, direction='forward')
    return merged[["bid", "ask", "bid_min_real", "bid_min_predicted", "bid_max_real", "bid_max_predicted", \
                   "ask_min_real", "ask_min_predicted", "ask_max_real", "ask_max_predicted"]]


# predicted=data[["bid","ask","bid_min_fut", "bid_max_fut", "ask_min_fut", "ask_max_fut"]]
# predicted.columns=[c.replace("_fut", "_predicted") for c in predicted.columns]
# predict_window=data["predict_window"].tail(1).values[0]
# real_min=data[["bid", "ask"]].rolling(predict_window).min().rename(columns={"bid":"bid_min_real", "ask":"ask_min_real"})
# real_max=data[["bid", "ask"]].rolling(predict_window).max().rename(columns={"bid":"bid_max_real", "ask":"ask_max_real"})
# real=pd.concat([real_min, real_max], axis=1)
# real["prediction_datetime"]=real.index.to_series()-pd.to_timedelta(predict_window)

# pd.merge_asof(real, predicted, left_on="prediction_datetime", right_index=True, direction='backward')


predictions = get_predictions(data)
predictions.info()
print(f"predict_window={data['predict_window'].values[-1]}")

## Prediction errors

In [ ]:
def plot_prediction_err(predictions):
    df = pd.DataFrame(index=predictions.index)
    for bidask in ["bid", "ask"]:
        for maxmin in ["max", "min"]:
            prefix = f"{bidask}_{maxmin}"
            df[f"{prefix}_error"] = predictions[f"{prefix}_predicted"] - predictions[f"{prefix}_real"]
    
    px.line(df, title="Errors", markers=True).show()
    px.line(df.sum(axis=1).abs().rename("error sum"), title="Error summary", markers=True).show()
    
plot_prediction_err(predictions)

In [ ]:
import pandas as pd

def get_profits(predictions: pd.DataFrame)->pd.DataFrame:
    """
    Buy, sell profit, real and predicted, for each time moment
    """
    
    profits = pd.DataFrame()
    # Profit/loss ratio to consider buy or sell
    min_ratio=1

    # Calculate possible profits for buy or sell
    profits["buy_profit_predicted"] = predictions["bid_max_predicted"] - predictions["ask"]
    profits["buy_loss_predicted"] = predictions["ask"]-predictions["bid_min_predicted"]
    profits['buy_ratio_predicted'] = np.where(profits['buy_loss_predicted'] > 0, profits['buy_profit_predicted'] / profits['buy_loss_predicted'],float('inf'))

    profits["buy_profit_real"] = predictions["bid_max_real"] - predictions["ask"]
    profits["buy_loss_real"] = predictions["ask"]-predictions["bid_min_real"]
    profits['buy_ratio_real'] = np.where(profits['buy_loss_real'] > 0, profits['buy_profit_real'] / profits['buy_loss_real'],float('inf'))


    profits["sell_profit_predicted"] = predictions["bid"] - predictions["ask_min_predicted"]
    profits["sell_loss_predicted"] = predictions["ask_max_predicted"] - predictions["bid"]
    profits['sell_ratio_predicted'] = np.where(profits['sell_loss_predicted'] > 0, profits['sell_profit_predicted'] / profits['sell_loss_predicted'],float('inf'))

    profits["sell_profit_real"] = predictions["bid"] - predictions["ask_min_real"]
    profits["sell_loss_real"] = predictions["ask_max_real"] - predictions["bid"]
    profits['sell_ratio_real'] = np.where(profits['sell_loss_real'] > 0, profits['sell_profit_real'] / profits['sell_loss_real'],float('inf'))


    profits["signal"] = np.where(profits['buy_ratio_predicted'] >= min_ratio, 1,
                                 np.where(profits["sell_ratio_predicted"]>=min_ratio, -1,0 ))

    # Calculate real profit or loss
    profits["profit_real"] = np.where(
        # If buy                             
        profits["signal"]==1,
        np.where(
            # Real movement down was inside what we predicted
            (predictions["bid_min_real"] < predictions["bid_min_predicted"]) 
            & (predictions["bid_max_real"] <= predictions["bid_max_predicted"]),
            # Take the profit
            profits["buy_profit_predicted"],
            # Or fix the loss
            -profits["buy_loss_predicted"]
        ),
        # If Sell                     
        np.where(profits["signal"] == -1,
            np.where(
                # Real movement down was inside what we predicted
                (predictions["ask_max_real"] > predictions["ask_max_predicted"]) 
                & (predictions["ask_min_real"] >= predictions["ask_max_predicted"]),
                # Take the profit
                profits["sell_profit_predicted"],
                # Or fix the loss
                -profits["sell_loss_predicted"]
            ),
            # If neither buy nor sell, return NaN
            np.nan
        )
    )

    # Fill in predicted profit
    profits["profit_predicted"]=np.where(profits["signal"] == 1, 
                                         # If buy
                                         profits["buy_profit_predicted"], 
                                        np.where(profits["signal"]==-1,
                                                # If sell
                                                profits["sell_profit_predicted"],
                                                # If neither buy not sell
                                                 np.nan))
    return profits

profits = get_profits(predictions)
pd.set_option('display.max_rows', 1000)
pd.concat([predictions,profits],axis=1).tail()
#profits.dropna().tail(100)




In [ ]:
print(f"Total prediction profit sum: {profits['profit_real'].sum()}")



In [ ]:
#px.line(profits[["profit_predicted", "profit_real"]], markers=True)
px.line(profits["profit_predicted"]-profits["profit_real"])

## Sharp ratio

In [ ]:
sharp_ratio_df=profits[["buy_ratio_predicted", "buy_ratio_real", "sell_ratio_predicted", "sell_ratio_real"]]
sharp_ratio_df.tail()


In [ ]:
px.line(sharp_ratio_df[["buy_ratio_predicted", "buy_ratio_real"]], title="Buy sharp ratio")

In [ ]:
px.line(sharp_ratio_df[["sell_ratio_predicted", "sell_ratio_real"]], title="Sell sharp ratio")

## Signal analysis

In [ ]:
y.tail()
y.plot(y= "open_signal", title="open signal")
plt.show()